NAATOS Bulk Analysis Tool for Modules
============

Simon Ghionea, Started 3/14/2025

## Imports


In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import xarray as xr
import math
import time

import plotly.express as px
import panel as pn

import naatos_module_tools.logreader as logreader
import naatos_module_tools.logprocessors as logprocessors
import naatos_module_tools.logplotter as logplotter

In [2]:
pn.extension("plotly")

# Enumerate, Select Experiment Folders, Load All Data

In [3]:
root = r'C:\Users\SimonGhionea\Global Health Labs, Inc\NAATOS Product Feasibility - General - Internal - Electronic Control Module\Beta design\PowermoduleTestData\by_exp'
#root = r'C:\Temp\NAATOS_MODULE_AUTODOWNLOADS'
rootpath = Path(root)

experiment_list = [x.name for x in rootpath.iterdir() if x.is_dir()];

experiments_to_plot = [
    #'20250128_sgdev_3.1',
    #'20250128_sgdev_3.1_b_abridge_pwmbugsearch',
    #'20250128_sgdev_3.1_c_abridge_diffpid',
    #'20250128_sgdev_3.1_d_2cycle_emulate',
    
    #'20250225_sgdev_cooldown_test',
    #'20250228_sgdev_cooldown_test',

    '20250311_DX_download',
    
    # '20250310_SimonGhionea_autodl',
    # '20250311_SimonGhionea_autodl',
    # '20250312_SimonGhionea_autodl',
    # '20250313_SimonGhionea_autodl_PM_eelab_continuous',
]

experiments_to_plot = [x for x in experiments_to_plot if x in experiment_list]

print('Found experiment folders:',experiments_to_plot)

Found experiment folders: ['20250311_DX_download']


In [4]:
#%% Load associated datafile from the unit-logged run
# logfile = logfilenames[0];
# df_in,df_events = logreader.scanALogfile(logfile)
dfraw = logreader.processRootFolder(rootpath,experiments_to_plot);
df_events = dfraw[ ~dfraw['Event'].isnull() & ~(dfraw['Event']==' ') ]

Processing rootfolder C:\Users\SimonGhionea\Global Health Labs, Inc\NAATOS Product Feasibility - General - Internal - Electronic Control Module\Beta design\PowermoduleTestData\by_exp
Experiment 20250311_DX_download
The folder we will assume each folder are UNITS
folder "Unit 12"
[]
Folder .
Done loading log sample_03-06-25_130105.csv
Done loading log sample_03-06-25_130105.csv
Done loading log sample_03-06-25_133331.csv
Done loading log sample_03-06-25_133331.csv
Done loading log sample_03-06-25_133432.csv
Done loading log sample_03-06-25_133432.csv
Done loading log sample_03-06-25_141509.csv
Done loading log sample_03-06-25_141509.csv
Done loading log sample_03-06-25_145851.csv
Done loading log sample_03-06-25_145851.csv
Done loading log sample_03-06-25_154038.csv
Done loading log sample_03-06-25_154038.csv
Done loading log sample_03-06-25_161655.csv
Done loading log sample_03-06-25_161655.csv
Done loading log sample_03-07-25_115333.csv
Done loading log sample_03-07-25_115333.csv
Done

# Pull Out Relevant Information

In [5]:
#%% Filter
df = dfraw;

#df = df[df['expname']=='20250311_SimonGhionea_autodl']
#df = df[df['unit']=='PM11'];
#df = df[df['run']=='sample_03-11-25_153808']
#df = df[df['run']=='sample_03-11-25_153400']

In [6]:
#%% Pull out by-cycle information

#dfbuildlist = [];
def convert_dtypes(df : pd.DataFrame):
    # try to convert strings to numerics as appropriate
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col]);
        except:
            pass
    
    return df

buildlist = [];
for unit,dfunits in df.groupby('unit'):
    #unit_run_counter = 0;
    for expname,dfexps in dfunits.groupby('expname'):
        for run,dfrun in dfexps.groupby('run'):
            #print('unit:{:s} exp:{:} run:{:s} shape:{:s}'.format(unit,expname,run,str(dfrun.shape)))
            
            dfrunevents = dfrun[ ~dfrun['Event'].isnull() & ~(dfrun['Event']==' ') ]
            dfrunevents = dfrunevents[['Time','Event']]
            
            status = 'nostatus';
            text = '';

            # Cycle Analysis
            evts_beg = dfrunevents[dfrunevents['Event'].str.match(r'^Cycle \d+ Started')]
            evts_end = dfrunevents[dfrunevents['Event'].str.match(r'^Cycle \d+ Stopped')]
            evts_nocyc = dfrunevents.loc[dfrunevents.index.symmetric_difference(evts_beg.index.tolist()+evts_end.index.tolist())]
            
            if( evts_beg.shape[0]==0 ):
                # was not a run
                #text+='Not A Run: {:s}'.format(str('\n'.join(dfrunevents['Event'].to_list())));
                text+='Not A Run: {:s}'.format(str(dfrunevents['Event'].to_list()));
                status = 'notrun';
                if(any(dfrunevents['Event'].str.startswith('Boot'))):
                    status = 'bootup';
                    
                    # check for boot reset reasons
                    reset_reason = dict( [tuple(s.split('=')) for s in dfrunevents['Event'].str.extract(r'POWER.RESETREAS=[^\s]*\s(.*)').dropna().iloc[0].item().split(' ')] );
                    # all reasons 0, I think that means power-on (turned on from switch) based on my reading of Nordic datasheet
                    if(all([v=='0' for k,v in reset_reason.items()])):
                        status+=',POR';
                
                dfrunevents['expname'] = expname;
                dfrunevents['unit'] = unit;
                dfrunevents['run'] = run;
                dfrunevents['status'] = status;
                dfrunevents['text'] = text;
                dfrunevents['Cycle'] = 0;
                dfrunevents = dfrunevents.rename(columns={'Time':'TimeBeg'})
                #buildlist.append(dfrunevents);
                buildlist.append( dfrunevents.drop(columns=['Event']).iloc[[0]] );
            
            elif( evts_beg.shape[0]==evts_end.shape[0]):
                #print('here5')
                # all Start and Stopped are matched
                ncycles = evts_beg.shape[0];
                cyc_num_beg = evts_beg['Event'].str.extract(r'Cycle (\d+) ',expand=False).astype(np.uint8);
                cyc_num_beg.name='Cycle'
                cyc_num_end = evts_end['Event'].str.extract(r'Cycle (\d+) ',expand=False).astype(np.uint8);
                cyc_num_end.name='Cycle'
                earlyterm = any(evts_end['Event'].str.match(r'.*early\.'));
                if(cyc_num_beg.shape[0]>0):
                    fn_lmbda_split_keyvalue_strings = lambda x: dict(tuple([tuple(z.split('=')) for z in x]));

                    #cyc_data_beg = evts_beg['Event'].str.extract(r'Cycle \d.*\.+( .*)',expand=False).str.split(' ').apply(lambda x: x[1:]).apply(lambda x: dict(tuple([tuple(z.split('=')) for z in x]))).apply(pd.Series)
                    #cyc_data_end = evts_end['Event'].str.extract(r'Cycle \d.*\.+( .*)',expand=False).str.split(' ').apply(lambda x: x[1:]).apply(lambda x: dict(tuple([tuple(z.split('=')) for z in x]))).apply(pd.Series)
                    cyc_data_beg = evts_beg['Event'].str.extract(r'Cycle \d.*\.+( .*)',expand=False).str.split(' ');
                    if(not all(cyc_data_beg.isna())):
                        # there are fields after "Cycle N started." messages
                        cyc_data_beg = cyc_data_beg.apply(lambda x: x[1:]).apply(fn_lmbda_split_keyvalue_strings).apply(pd.Series)
                        cyc_data_beg = convert_dtypes(cyc_data_beg);

                        evts_beg = pd.concat((evts_beg,cyc_num_beg,cyc_data_beg),axis='columns')
                        #print('here1');
                    else:
                        evts_beg = pd.concat((evts_beg,cyc_num_beg),axis='columns');
                        #print('here2');
                    evts_beg = evts_beg.rename(columns={'Time':'TimeBeg'})


                    cyc_data_end = evts_end['Event'].str.extract(r'Cycle \d.*\.+( .*)',expand=False).str.split(' ');
                    if(not all(cyc_data_end.isna())):
                        # there are fields after "Cycle N started." messages
                        cyc_data_end = cyc_data_end.apply(lambda x: x[1:]).apply(fn_lmbda_split_keyvalue_strings).apply(pd.Series)
                        cyc_data_end = convert_dtypes(cyc_data_end);

                        evts_end = pd.concat((evts_end,cyc_num_end,cyc_data_end),axis='columns')
                        #print('here3')
                    else:
                        evts_end = pd.concat((evts_end,cyc_num_end),axis='columns');
                        #print('here4');
                    evts_end = evts_end.rename(columns={'Time':'TimeEnd'})

                    # Combine Beginning and Ending of cycle information into a single cycle row
                    cyc_data = pd.merge(evts_beg,evts_end,on='Cycle',suffixes=('Beg','End')).set_index('Cycle',drop=True);

                    # calculate cycle information
                    cyc_data['CCycleRTCSeconds'] = (cyc_data['TimeEnd']-cyc_data['TimeBeg']).apply(lambda x: x.total_seconds());
                    #cyc_data.loc[0,['EventEnd']] = 'norm'
                else:
                    print('\tOTHER')

                
                if(earlyterm):                   
                    # print('\tEarly abort in cycle {:d} @ {:.0f} s @ total runtime {:.0f} s:'.format(
                    #     cyc_data.index[-1],
                    #     cyc_data['CalcCycleRTCSeconds'].iloc[-1].item(),
                    #     cyc_data['CalcCycleRTCSeconds'].sum().item()
                    #     )
                    # )
                    text += 'Early abort in cycle {:d} @ {:.0f} s @ total runtime {:.0f} s:\n'.format(
                        cyc_data.index[-1],
                        cyc_data['CCycleRTCSeconds'].iloc[-1].item(),
                        cyc_data['CCycleRTCSeconds'].sum().item()
                    );
                    #print('\t',str(evts_nocyc['Event'].to_list()))
                    #text += '{:s}'.format('\n'.join(evts_nocyc['Event'].to_list()));
                    text += '{:s}'.format(str(evts_nocyc['Event'].to_list()));
                    normal_interruptions = [
                        'HALL sensor interrupted',
                        'Optical sensor interrupted',
                        'ButtonCycle cancled via button click',
                    ]
                    if(any(pd.concat([evts_nocyc['Event'].str.contains(s) for s in normal_interruptions]))):
                        status = 'run_ended_early_user';
                    else:
                        status = 'run_ended_early_other';
                    # if(evts_nocyc['Event'].str.contains('Optical sensor interrupted')|evts_nocyc['Event'].str.contains('Hall sensor interrupted')):
                    #     status = 'run_ended_early';
                else:
                    # print('\tCompletedRun @ total runtime {:.0f} s'.format(
                    #     cyc_data['CalcCycleRTCSeconds'].sum().item()
                    # ))
                    text += 'CompletedRun @ total runtime {:.0f}s:'.format(
                        cyc_data['CCycleRTCSeconds'].sum().item()
                    )
                    #text += '{:s}'.format('\n'.join(evts_nocyc['Event'].to_list()));
                    text += '{:s}'.format(str(evts_nocyc['Event'].to_list()));
                    status = 'run_success';
                
                try:
                    if(not earlyterm):
                        if( not all(cyc_data_beg['runtime_s']==cyc_data_end['expected_sec']) ):
                            #print('\tMismatched runtimes');
                            text += '\n\tMismatched runtimes';
                            status = 'run_mismatched_runtimes';
                    else:
                        if( not all(cyc_data_beg['runtime_s'][0:-1]==cyc_data_end['expected_sec'][0:-1]) ):
                            #print('\tMismatched runtimes (early)');
                            text += '\n\tMismatched runtimes (early)';
                            status = 'run_mismatched_runtimes_early';
                except:
                    pass;
                
                # SUMMARIZE PER RUN
                if(not earlyterm):
                    pass;
                    # for (k1,v1),(k2,v2) in zip(evts_beg.iterrows(),evts_end.iterrows()):
                    #     #print( 'Cycle{:d} idx{:d} to idx{:d}'.format(v1['Cycle'],k1,k2) );
                        
                    #     # subset this cycle in the run
                    #     dfruncyc = dfrun.loc[k1:k2]

                    #     # last third
                    #     dfruncyclst3rd = dfruncyc.loc[k2-((k2-k1)//3):]

                    #     # calculate some summaries
                    #     #cyc_data.loc[v1['Cycle'],'C'] = cyc_data['CCycleRTCSeconds'].sum().item()
                    #     cyc_data.loc[v1['Cycle'],'CL3MedAmpTemp'] = dfruncyclst3rd['AmpTemp'].median()
                    #     cyc_data.loc[v1['Cycle'],'CL3MedValveTemp'] = dfruncyclst3rd['ValveTemp'].median()
                    #     cyc_data.loc[v1['Cycle'],'CL3MaxAmpPWM'] = dfruncyclst3rd['AmpPWM'].max()
                    #     cyc_data.loc[v1['Cycle'],'CL3MaxValvePWM'] = dfruncyclst3rd['ValvePWM'].max()
                    #     cyc_data.loc[v1['Cycle'],'CMaxAmpTemp'] = dfruncyc['AmpTemp'].max()
                    #     cyc_data.loc[v1['Cycle'],'CMaxValveTemp'] = dfruncyc['ValveTemp'].max()

                    #     pass
                for (k1,v1),(k2,v2) in zip(evts_beg.iterrows(),evts_end.iterrows()):
                    #print( 'Cycle{:d} idx{:d} to idx{:d}'.format(v1['Cycle'],k1,k2) );
                    
                    # subset this cycle in the run
                    dfruncyc = dfrun.loc[k1:k2]

                    # last third
                    dfruncyclst3rd = dfruncyc.loc[k2-((k2-k1)//3):]

                    # calculate some summaries
                    #cyc_data.loc[v1['Cycle'],'C'] = cyc_data['CCycleRTCSeconds'].sum().item()
                    #cyc_data.loc[v1['Cycle'],'C'] = cyc_data['CCycleRTCSeconds'].sum().item()
                    cyc_data.loc[v1['Cycle'],'CL3MedAmpTemp'] = dfruncyclst3rd['AmpTemp'].median()
                    cyc_data.loc[v1['Cycle'],'CL3MedValveTemp'] = dfruncyclst3rd['ValveTemp'].median()
                    cyc_data.loc[v1['Cycle'],'CL3MaxAmpPWM'] = dfruncyclst3rd['AmpPWM'].max()
                    cyc_data.loc[v1['Cycle'],'CL3MaxValvePWM'] = dfruncyclst3rd['ValvePWM'].max()
                    cyc_data.loc[v1['Cycle'],'CMaxAmpTemp'] = dfruncyc['AmpTemp'].max()
                    cyc_data.loc[v1['Cycle'],'CMaxValveTemp'] = dfruncyc['ValveTemp'].max()

                    pass
                #cyc_data['CRTCRuntime'] = (dfrun['Time'].iloc[[0,-1]]).diff().apply(lambda x: x.total_seconds()).iloc[-1].item();
                cyc_data['CRTCRuntime'] = ( cyc_data.iloc[-1]['TimeEnd'] - cyc_data.iloc[0]['TimeBeg'] ).total_seconds();

                cyc_data['expname'] = expname;
                cyc_data['unit'] = unit;
                cyc_data['run'] = run;
                cyc_data['status'] = status;
                cyc_data['text'] = text;
                buildlist.append(cyc_data.reset_index());
            else:
                text += 'Run cycles non-sensical, nbegun={:d} nended={:d}'.format(evts_beg.shape[0],evts_end.shape[0]);
                status = 'nonsensical_cycles';

                dfrunevents['expname'] = expname;
                dfrunevents['unit'] = unit;
                dfrunevents['run'] = run;
                dfrunevents['status'] = status;
                dfrunevents['text'] = text;
                dfrunevents['Cycle'] = 0;
                dfrunevents.rename(columns={'Time':'TimeBeg'});
                buildlist.append(dfrunevents);

            if(status == 'notrun'):
                print('unit:{:s} exp:{:} run:{:s} shape:{:s} status:{:s}'.format(unit,expname,run,str(dfrun.shape),status))
                print(text)
            #break;


#dflifetest = pd.concat(dfbuildlist)
dfbuilt = pd.concat(buildlist,ignore_index=True)
dfbuilt['Cycle'] = dfbuilt['Cycle'].astype(np.uint8)
dfbuilt = dfbuilt.set_index(['unit','expname','run','Cycle'])
#dfbuilt = pd.concat(buildlist).reset_index()

#%% Show run summary

# UI Playing

In [ ]:
#%% Output Summary Table By Unit

mydf = dfbuilt.reorder_levels(['expname','unit','run','Cycle']);
# mydf = dfbuilt.reset_index().set_index(['expname','unit']);
# mydf = mydf[['run','Cycle','TimeBeg','TimeEnd','status','text']];
#mydf = dfbuilt.reset_index().set_index(['expname','unit']);
#mydf = dfbuilt.reset_index()
#mydf = mydf[['run','Cycle','TimeBeg','status','text']];

#mydf.apply(lambda x: '<a href="file:///c:\\TEMP\\runtime_{:s}_{:s}_{:s}.html">link</a>'.format( x['exp'],'b','c' ),axis=1)
def mkrunlink(x):
    if((x['status'] == 'run_success') or (x['status'].find('run_ended_early')>=0)):
        return '<a href="file:///c:\\TEMP\\NAATOS_PM_RUN_runtime_{:s}_{:s}_{:s}.html" target="_blank">{:s}</a>'.format( x['expname'],x['unit'],x['run'],x['run'] );
    else:
        return x['run'];
    #return 'test';
ret = dfbuilt.reset_index().apply( lambda x: mkrunlink(x) ,axis=1);
mydf['run'] = ret.tolist();

# simplify Time field
#ret = mydf.reset_index().apply( lambda x: x['TimeBeg'] if pd.isna(x['Time'])==True else x['Time'],axis=1);
#mydf['Time'] = ret.tolist();

# Choose Columns
mydf = mydf[['status','TimeBeg',*dfbuilt.columns[dfbuilt.columns.str.startswith('C')].tolist(),'text']];
#mydf = mydf[mydf['status']=='notrun'];
mydf = mydf.reset_index();
mydf = mydf.sort_values(['expname','TimeBeg','Cycle'])

# STYLING
def style_statustext(val):
    """
    Takes a scalar and returns a string with
    the css property
    """
    if(val=='run_success'):
        return "background-color: lightgreen";
    elif(val=='run_ended_early_user'):
        return "background-color: lightyellow";
    elif(val=='notrun'):
        return '';
    elif(val.find('boot')>=0):
        return "background-color: powderblue";
    else:
        return "background-color: orchid";
mydf_styled = mydf.style.map(style_statustext,subset=pd.IndexSlice[:, ['status']]);

header_filters = {
    'text': {'type': 'input', 'func': 'like', 'placeholder': 'Search text'},
}
tabulator_editors = {
    'unit': {'type': 'list', 'valuesLookup': True},
    'status': {'type': 'list', 'valuesLookup': True},
    'text': {'type': 'input', 'func': 'like', 'placeholder': 'Search text'},
}
pn_summarytable = pn.widgets.Tabulator(
    value=mydf_styled,

    pagination='local', page_size=30,
    #pagination='local',page_size=1000,

    #hierarchical=True,
    #groupby=['expname','unit'],
    #theme='midnight',
    #aggregators={"origin": "mean", "yr": "mean"},
    
    formatters = {
        'run': dict(type='html'),
        'text': dict(type='textarea'),
    },

    #header_filters=header_filters,
    editors=tabulator_editors,header_filters=True,

    configuration=dict(
        paginationCounter="rows"
    ),
    
    sizing_mode='stretch_both',
);
# pn_summarytable = pn.widgets.DataFrame(
#     value=mydf,
#     hierarchical=True,
#     sizing_mode='stretch_both', width_policy='max',autosize_mode='fit_viewport'
# );

pn_final = pn.Column(pn_summarytable).servable();
#pn_final.save(r'C:\TEMP\summaryinfo.html');
#pn_final
pn.serve(pn_final);

In [ ]:
mydf[mydf['status'].str.startswith('boot')]

# Panel Multi-Page Setup

In [ ]:
def page1():
    return "page 1"

def page2():
    return pn.Column(
        "# Page 2", "Welcome to the second page"
    )
ROUTES = {
    "1": page1, "2": page2
}
pn.serve(ROUTES, port=5006)

# Parameterized

In [58]:
dffull = dfbuilt.reset_index().copy();
dffullorig = dffull.copy(); #<-- store a copy so we can come back to this
dfsel = pd.DataFrame();

In [59]:
# %% Helpers
def filtered_dataframe(param_values,param_columns):
    
    selection_dict = pd.Series(param_values)[list(param_columns.keys())].to_dict();
    print('selection_dict',selection_dict);

    # this line will filter the dataframe using param_values and param_columns
    #dffiltered = dffull.isin(pd.Series(obj.param.values())[param_columns.keys()].to_dict())[param_columns].all(axis=1);
    #dffiltered = dffull[dffull.isin(pd.Series(param_values)[param_columns.keys()].to_dict())[param_columns].all(axis=1)]

    dffiltered = dffull[dffull.isin(selection_dict)[param_columns.keys()].all(axis=1)]
    
    return dffiltered;

In [ ]:
import param
class ExplorerApp(param.Parameterized):
    data = param.DataFrame(doc="Stores a DataFrame to explore")

    columns = param.ListSelector(
        default=["unit","expname","run","Cycle","TimeBeg","status","text"]
    )

    filtered_data = param.DataFrame(doc="Stores the filtered DataFrame");

    actionbutton = param.Action(lambda x: x.param.trigger('actionbutton'), label='Update table');
    refinebutton = param.Action(lambda x: x.param.trigger('refinebutton'), label='Refine selection');
    resetchoices = param.Action(lambda x: x.param.trigger('resetchoices'), label='Reset choices');

    _widget_tabulator = pn.widgets.Tabulator(
        name='DataFrame',
        selectable='checkbox',
        disabled=True, # non-editable data
        show_index=False,
        layout='fit_data', # auto-fit
        #pagination = None,
        pagination = 'remote',
        page_size=25,
    );

    _pane_main = pn.pane.Str('test');
    _pane_idx = None;

    def _cb_select_all(self,event, tabulator_widget, selectAll=True):
        if(selectAll):
            tabulator_widget.selection = list(range(len(tabulator_widget.value)));
        else:
            tabulator_widget.selection = [];

    def __init__(self, **kwargs):
        super().__init__(**kwargs); # MUST CALL SUPER constructor for panel to work

        # define columns
        self.param.columns.objects = dffull.columns.to_list()

        # add dataframe interactivity
        dfrx = self.param.data.rx();

        # filtered dataframe
        #self.filtered_data = dfrx[self.param.columns];


        # self._pane_main = pn.Tabs(
        #     #('idx',self._widget_tabulator),
        #     ('idx',self._pane_idx),
        #     ('gallery','TEST2!'),
        #     ('detail','TEST3!'),
        #     dynamic=True,
        #     tabs_location='above',
        # );

    @param.depends("columns", watch=True, on_init=True)
    def _update_filtered_data(self):
        df = self.data
        self.filtered_data=df[
            self.columns
        ]

    @param.depends('data')
    def _mk_tabulator_widget(self):
        def mkrunlink(x):
            if((x['status'] == 'run_success') or (x['status'].find('run_ended_early')>=0)):
                return '<a href="file:///c:\\TEMP\\NAATOS_PM_RUN_runtime_{:s}_{:s}_{:s}.html" target="_blank">{:s}</a>'.format( x['expname'],x['unit'],x['run'],x['run'] );
            else:
                return x['run'];
            #return 'test';
        mydf = self.filtered_data;
        ret = mydf.reset_index().apply( lambda x: mkrunlink(x) ,axis=1);
        mydf['run'] = ret.tolist();

        # simplify Time field
        #ret = mydf.reset_index().apply( lambda x: x['TimeBeg'] if pd.isna(x['Time'])==True else x['Time'],axis=1);
        #mydf['Time'] = ret.tolist();

        # Choose Columns
        #mydf = mydf[['status','TimeBeg',*dfbuilt.columns[dfbuilt.columns.str.startswith('C')].tolist(),'text']];
        #mydf = mydf[mydf['status']=='notrun'];
        mydf = mydf.reset_index();
        mydf = mydf.sort_values(['expname','TimeBeg','Cycle'])

        # STYLING
        def style_statustext(val):
            """
            Takes a scalar and returns a string with
            the css property
            """
            if(val=='run_success'):
                return "background-color: lightgreen";
            elif(val=='run_ended_early_user'):
                return "background-color: lightyellow";
            elif(val=='notrun'):
                return '';
            elif(val.find('boot')>=0):
                return "background-color: powderblue";
            else:
                return "background-color: orchid";
        mydf_styled = mydf.style.map(style_statustext,subset=pd.IndexSlice[:, ['status']]);

        header_filters = {
            'text': {'type': 'input', 'func': 'like', 'placeholder': 'Search text'},
        }
        tabulator_editors = {
            'unit': {'type': 'list', 'valuesLookup': True},
            'status': {'type': 'list', 'valuesLookup': True},
            'text': {'type': 'input', 'func': 'like', 'placeholder': 'Search text'},
        }
        pn_summarytable = pn.widgets.Tabulator(
            value=mydf_styled,

            pagination='local', page_size=30,
            #pagination='local',page_size=1000,

            #hierarchical=True,
            #groupby=['expname','unit'],
            #theme='midnight',
            #aggregators={"origin": "mean", "yr": "mean"},
            
            formatters = {
                'run': dict(type='html'),
                'text': dict(type='textarea'),
            },

            #header_filters=header_filters,
            editors=tabulator_editors,header_filters=True,

            configuration=dict(
                paginationCounter="rows"
            ),
            
            sizing_mode='stretch_both',
        );
        return pn_summarytable;

    @param.depends('refinebutton', watch=True)
    def _refine_choices(self):
        global dffull;
        #dffullorig = dffull.copy(); # keep original, as we will be filtering dffull later
        #print('Refine choices');
        #print(self.param.values());
        #print(param_columns);
        dffull = filtered_dataframe(self.param.values(), param_columns);
        print('Refine choices dffull.shape',dffull.shape);
        update_or_add_parameters(param_columns);
        #self._widget_tabulator.value = dffull.drop(columns=['fyuv','fjpg']);
        self._widget_tabulator.value = dffull;
        #print(self.param['ctrl_ghl_agc']);
        
    @param.depends('resetchoices', watch=True)
    def _reset_choices(self):
        global dffull;
        dffull = dffullorig.copy();
        update_or_add_parameters(param_columns);

    # callback - fires when the button is clicked
    @param.depends('actionbutton', watch=True)
    def _update_figure(self):
        global dffull;
        print('_update_figure()');

        self._refine_choices();

        #self.build_gallery();
        print('Update Figure')

    # # callback - fires when dataframe selection is edited
    # @param.depends('_widget_tabulator.selection', watch=True)
    # def _tabulator_selection_change(self):
    #     print('df selection',self._widget_tabulator.selection);

    #
    def __panel__(self):
        pn_but_idx_selall = pn.widgets.Button(name='Select All', button_type='primary');
        pn_but_idx_selall.on_click(lambda event: self._cb_select_all(event,self._widget_tabulator,True));
                                   
        pn_but_idx_selnone = pn.widgets.Button(name='Select None', button_type='primary');
        # pn_but_idx_selall.on_click(lambda event:
        #     setattr(self._widget_tabulator,'selection',[]),
        # )
        pn_but_idx_selnone.on_click(lambda event: self._cb_select_all(event,self._widget_tabulator,False));


        return pn.Column(
                pn.Row(pn_but_idx_selall,pn_but_idx_selnone),
                #self._widget_tabulator,
                self._mk_tabulator_widget()
            )

#%% Panel creation
# which columns will we use?
param_columns = {
    'expname':param.ListSelector,
    'unit':param.ListSelector,
}
# # all control columns
# for cname in dffull.columns[dffull.columns.str.startswith('ctrl_')].tolist():
#     param_columns[cname] = param.ListSelector;

# create the panel object
obj = ParametricRawViewApp(data=dffull);
def update_or_add_parameters(param_columns):
    for col,paramobj in param_columns.items():
        items = dffull[col].unique().tolist();
        
        if col not in obj.param:
            # add parameter
            newParam = paramobj(default=items,objects=items);
            print('Adding param',col,'dffull.shape',dffull.shape)
            obj.param.add_parameter(col,newParam);
        else:
            # update parameter
            print('Updating param',col,'dffull.shape',dffull.shape)
            # reset available options
            obj.param[col].objects = items;
            # reset selections (to all)
            setattr(obj,col,items);
update_or_add_parameters(param_columns);

# format params, customize widgets
# formatted_params = pn.Param(obj.param, widgets={
#         'stimstr': {'height':250},
#         'surfgen': {'height':50},
#         'opt_method': {'height':50},
#         'opt_measured_v_i': {'height':50},
#         #'mdln': {'height':50},
#         's_spec': {'height':50},
#     },
#     width=300
# )
formatted_params = pn.Param(obj.param,
    width=300,
    widgets={"columns": pn.widgets.MultiChoice}
)

#%% Final
pn_final = pn.Row(
    #obj.param, obj.view,
    formatted_params, obj
    #width_policy='max',
    #height_policy='max',
).servable("GHL NAATOS Module Logs Explorer")
pn.serve(pn_final)

In [ ]:
obj.data

# Parameterized 2

In [60]:
dffull = dfbuilt.reset_index().copy();
dffullorig = dffull.copy(); #<-- store a copy so we can come back to this
dfsel = pd.DataFrame();

In [61]:
# %% Helpers
def filtered_dataframe(param_values,param_columns):
    
    selection_dict = pd.Series(param_values)[list(param_columns.keys())].to_dict();
    print('selection_dict',selection_dict);

    # this line will filter the dataframe using param_values and param_columns
    #dffiltered = dffull.isin(pd.Series(obj.param.values())[param_columns.keys()].to_dict())[param_columns].all(axis=1);
    #dffiltered = dffull[dffull.isin(pd.Series(param_values)[param_columns.keys()].to_dict())[param_columns].all(axis=1)]

    dffiltered = dffull[dffull.isin(selection_dict)[param_columns.keys()].all(axis=1)]
    
    return dffiltered;

In [82]:
from panel.viewable import Viewer
import param

data_url = "https://assets.holoviz.org/panel/tutorials/turbines.csv.gz"
#turbines = pn.cache(pd.read_csv)(data_url)

c_def_cols = ["unit","expname","run","Cycle","TimeBeg","status","text"];
c_filters = {
    "expname":param.ListSelector,
    "unit":param.ListSelector,
    "status":param.ListSelector,
    "Cycle":param.ListSelector,
    "TimeBeg":param.DateRange
};

def update_or_add_parameters(obj,param_columns):
    for col,paramobj in param_columns.items():
        items = dffull[col].unique().tolist();
        
        if(paramobj == param.ListSelector):
            if col not in obj.param:
                # add parameter
                newParam = paramobj(default=sorted(items),objects=sorted(items));
                print('Adding param',col,'dffull.shape',dffull.shape)
                obj.param.add_parameter(col,newParam);
            else:
                # update parameter
                print('Updating param',col,'dffull.shape',dffull.shape)
                # reset available options
                obj.param[col].objects = items;
                # reset selections (to all)
                setattr(obj,col,items);
        elif(paramobj == param.DateRange):
            print('DateRange')
            if col not in obj.param:
                # add parameter
                beg = sorted(dffull['TimeBeg'].unique())[0];
                end = sorted(dffull['TimeBeg'].unique(),reverse=True)[0];
                newParam = paramobj(default=(beg,end),bounds=(beg,end));
                print('Adding param',col,'dffull.shape',dffull.shape)
                obj.param.add_parameter(col,newParam);

class DataExplorer(Viewer):
    data = param.DataFrame(doc="Stores a DataFrame to explore")

    columns = param.ListSelector(
        #default=["p_name", "t_state", "t_county", "p_year", "t_manu", "p_cap"]
        default=c_def_cols,
    )

    #year = param.Range(default=(1981, 2022), bounds=(1981, 2022))
    #capacity = param.Range(default=(0, 1100), bounds=(0, 1100))

    filtered_data = param.DataFrame(doc="Stores the filtered DataFrame")

    def __init__(self, **params):
        super().__init__(**params)
        
        # set parameter for columns to those from dataframe
        self.param.columns.objects = self.data.columns.to_list()

        # dynamically add parameters
        update_or_add_parameters(self,c_filters);

    @param.depends("data", "columns", watch=True, on_init=True)
    def _update_filtered_data(self):
        df = self.data
        # self.filtered_data=df[df.p_year.between(*self.year) & df.p_cap.between(*self.capacity)][
        #     self.columns
        # ]
    

        # after filtering
        df = df[ self.columns ];
    
        # styler
        # # STYLING
        # def style_statustext(val):
        #     """
        #     Takes a scalar and returns a string with
        #     the css property
        #     """
        #     if(val=='run_success'):
        #         return "background-color: lightgreen";
        #     elif(val=='run_ended_early_user'):
        #         return "background-color: lightyellow";
        #     elif(val=='notrun'):
        #         return '';
        #     elif(val.find('boot')>=0):
        #         return "background-color: powderblue";
        #     else:
        #         return "background-color: orchid";
        # mydf_styled = df.style.map(style_statustext,subset=pd.IndexSlice[:, ['status']]);
    
        # some formatting
        def mkrunlink(x):
            if((x['status'] == 'run_success') or (x['status'].find('run_ended_early')>=0)):
                return '<a href="file:///c:\\TEMP\\NAATOS_PM_RUN_runtime_{:s}_{:s}_{:s}.html" target="_blank">{:s}</a>'.format( x['expname'],x['unit'],x['run'],x['run'] );
            else:
                return x['run'];
            #return 'test';
        ret = df.apply( lambda x: mkrunlink(x) ,axis=1);
        df['run'] = ret.tolist();

        self.filtered_data = df;


    @param.depends('filtered_data')
    def number_of_rows(self):
        return f"Rows: {len(self.filtered_data)}"
    
    def _tabulatorWidget(self):
        print('_tabulatorWidget called');

        header_filters = {
            'text': {'type': 'input', 'func': 'like', 'placeholder': 'Search text'},
        }
        tabulator_editors = {
            'unit': {'type': 'list', 'valuesLookup': True},
            'status': {'type': 'list', 'valuesLookup': True},
            'text': {'type': 'input', 'func': 'like', 'placeholder': 'Search text'},
        }
        pn_summarytable = pn.widgets.Tabulator(
            value=self.param.filtered_data,

            pagination='local', page_size=30,
            #pagination='local',page_size=1000,

            #hierarchical=True,
            #groupby=['expname','unit'],
            #theme='midnight',
            #aggregators={"origin": "mean", "yr": "mean"},
            show_index=False,
            formatters = {
                'run': dict(type='html'),
                'text': dict(type='textarea'),
            },

            #header_filters=header_filters,
            editors=tabulator_editors,header_filters=True,

            configuration=dict(
                paginationCounter="rows"
            ),
            
            sizing_mode='stretch_both',
        );

        # apply dataframe styling to the tabulator widget
        # https://discourse.holoviz.org/t/dynamic-update-of-tabulator-style/2741
        # STYLING
        def style_statustext(val):
            """
            Takes a scalar and returns a string with
            the css property
            """
            if(val=='run_success'):
                return "background-color: lightgreen";
            elif(val=='run_ended_early_user'):
                return "background-color: lightyellow";
            elif(val=='notrun'):
                return '';
            elif(val.find('boot')>=0):
                return "background-color: powderblue";
            else:
                return "background-color: orchid";
        mydf_styled = pn_summarytable.style.map(style_statustext,subset=pd.IndexSlice[:, ['status']]);


        return pn_summarytable;

    def __panel__(self):
        print('__panel__() method called')
        c_display = set(obj.param.values().keys())-set(('columns','data','filtered_data','name'));
        
        return pn.Column(
            pn.Row(
                pn.Column(
                    pn.widgets.MultiChoice.from_param(self.param.columns, width=400),
                ),
                #pn.Column(self.param.year, self.param.capacity),
                *[obj.param[x] for x in c_display]
            ),
            self.number_of_rows,
            
            #pn.widgets.Tabulator(self.param.filtered_data, page_size=10, pagination="remote"),
            self._tabulatorWidget()
        )
#s = DataExplorer(data=turbines).servable()
obj = DataExplorer(data=dffull)
#s = obj.servable();


Adding param expname dffull.shape (493, 19)
Adding param unit dffull.shape (493, 19)
Adding param status dffull.shape (493, 19)
Adding param Cycle dffull.shape (493, 19)
DateRange
Adding param TimeBeg dffull.shape (493, 19)


C:\Users\SimonGhionea\AppData\Local\Temp\ipykernel_20548\3170679435.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['run'] = ret.tolist();


In [ ]:
pn.serve(s)

In [ ]:
set(obj.param.values().keys())-set(('columns','data','filtered_data','name'))

In [ ]:
sorted(dffull['TimeBeg'].unique())[0]

In [ ]:
dir(obj.filtered_data)

# Plotting Panel

In [83]:
class Plotter1App(Viewer):
    dffull = param.DataFrame(doc="Stores the master dataframe")
    dfbuilt = param.DataFrame(doc="Stores the processed cycle run")

    expname = param.String();
    unit = param.String();
    run = param.String();

    def __init__(self,**params):
        # get first info and set as default property
        expname,unit,run = list(dffull.groupby(['expname','unit','run']).groups.keys())[0];
        #print(expname,unit,run);
        self.param.expname.default = expname;
        self.param.unit.default = unit;
        self.param.run.default = run;
    
    def __panel__(self):
        # return pn.Column(
        #     pn.Row(
        #         pn.Column(
        #             pn.widgets.MultiChoice.from_param(self.param.columns, width=400),
        #         ),
        #         #pn.Column(self.param.year, self.param.capacity),
        #         pn.
        #     ),
            
        #     #pn.widgets.Tabulator(self.param.filtered_data, page_size=10, pagination="remote"),
        #     self._tabulatorWidget()

        # pn.Column(

        # )
        print('2 __panel__');
        #pn.state.location.sync(obj2,['unit'])
        print('2 __panel__',pn.state.location);
        return pn.Column(
            pn.Row(
                self.param.expname,self.param.unit,self.param.run
            ),
            'PLOT GOES HERE'
        );
obj2 = Plotter1App(dffull = dffull, dfbuilt = dfbuilt);


# Panel Multi-Page Setup 2

In [ ]:
def page1():
    return obj

def page2():
    #pn.state.location.sync(obj2,['unit']);
    return obj2
ROUTES = {
    "": page1,
    "2": page2
}
# if pn.state.location:
#     pn.state.location.sync(obj2)
serve = pn.serve(ROUTES, port=5006)

Launching server at http://localhost:5006


__panel__() method called
_tabulatorWidget called
2 __panel__
2 __panel__ <Location Location03331>
2 __panel__
2 __panel__ <Location Location03358>
2 __panel__
2 __panel__ <Location Location03385>


In [81]:
serve.stop()

AssertionError: Already stopped

In [88]:
pn.state.location

# OTher


In [ ]:
#mydf.to_excel(r'C:\temp\summarytable.xlsx',freeze_panes=(1,1))

In [ ]:
#list(dfraw.groupby(['expname','unit','run']).groups.keys())[0]